## Az eredményeket mentsd a src/weekly modul-ba weekly_test_6.py néven

#### Használható modulok: pathlib, pandas, typing, str, statsmodels

In [1]:
from pathlib import Path

In [2]:
datalib = Path.cwd().parent.joinpath('data')

In [ ]:
"""
1., Olvasd be a data mappa sp500.parquet nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet
"""

In [23]:
import pandas as pd

df_sp500 = pd.read_parquet('../data/sp500.parquet', engine='fastparquet')

df_sp500.head()

,Symbol,Monthly Returns
Date,,
2000-02-29,A,0.569405
2000-03-31,A,0.001203
2000-05-31,A,-0.169252
2000-06-30,A,0.001698
2000-07-31,A,-0.447458


In [ ]:
"""
2., Olvasd be az ff_factors.parquet fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet
"""

In [25]:
import pandas as pd

df_ff_factors = pd.read_parquet('../data/ff_factors.parquet', engine='fastparquet')

df_ff_factors.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-31,0.0296,-0.0256,-0.0243,0.022
1926-08-31,0.0264,-0.0117,0.0382,0.025
1926-09-30,0.0036,-0.0140,0.0013,0.023
1926-10-31,-0.0324,-0.0009,0.0070,0.032
1926-11-30,0.0253,-0.0010,-0.0051,0.031


In [ ]:
"""
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a 'Date' elsődleges kulcs alapján.
"""

![Joined data](../resources/weekly6/joined_data.jpg)

In [27]:
merged_df = pd.merge(df_sp500, df_ff_factors, on='Date', how='left')

merged_df.head()

,Symbol,Monthly Returns,Mkt-RF,SMB,HML,RF
Date,,,,,,
2000-02-29,A,0.569405,0.0245,0.2136,-0.0959,0.043
2000-03-31,A,0.001203,0.0520,-0.1720,0.0813,0.047
2000-05-31,A,-0.169252,-0.0442,-0.0605,0.0475,0.050
2000-06-30,A,0.001698,0.0464,0.1284,-0.0842,0.040
2000-07-31,A,-0.447458,-0.0251,-0.0307,0.0831,0.048


In [ ]:
"""
4., Készíts egy új 'Excess Return' nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
"""

In [28]:
merged_df['Excess Return'] = merged_df['Monthly Returns'] - merged_df['RF']

merged_df.head()

,Symbol,Monthly Returns,Mkt-RF,SMB,HML,RF,Excess Return
Date,,,,,,,
2000-02-29,A,0.569405,0.0245,0.2136,-0.0959,0.043,0.526405
2000-03-31,A,0.001203,0.0520,-0.1720,0.0813,0.047,-0.045797
2000-05-31,A,-0.169252,-0.0442,-0.0605,0.0475,0.050,-0.219252
2000-06-30,A,0.001698,0.0464,0.1284,-0.0842,0.040,-0.038302
2000-07-31,A,-0.447458,-0.0251,-0.0307,0.0831,0.048,-0.495458


In [ ]:
"""
5., Rendezd sorba dátum szerint az adatokat, majd generálj egy új oszlopot ('ex_ret_1'), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a következő időszaki Excess Return érték. 
"""

![new column](../resources/weekly6/ex_ret_1.jpg)

In [29]:
merged_df = merged_df.sort_values(by=['Date', 'Symbol'])

merged_df['ex_ret_1'] = merged_df.groupby('Symbol')['Excess Return'].shift(-1)

merged_df.head()

,Symbol,Monthly Returns,Mkt-RF,SMB,HML,RF,Excess Return,ex_ret_1
Date,,,,,,,,
2000-01-31,AAPL,-0.938186,-0.0474,0.0577,-0.0188,0.041,-0.979186,0.061820
2000-01-31,ABT,-0.900699,-0.0474,0.0577,-0.0188,0.041,-0.941699,-0.029564
2000-01-31,ACGL,-0.984903,-0.0474,0.0577,-0.0188,0.041,-1.025903,-0.000265
2000-01-31,ADBE,-0.947964,-0.0474,0.0577,-0.0188,0.041,-0.988964,0.809441
2000-01-31,ADI,-0.911564,-0.0474,0.0577,-0.0188,0.041,-0.952564,0.638818


In [ ]:
"""
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer törlöd az össze olyan sort, amely az 'ex_ret_1' oszlopban hiányos, majd ezt követően, törlöd az összes olyan sort, ami a 'HML' oszlopban hiányos
"""

In [30]:
merged_df = merged_df.sort_values(by=['Date', 'Symbol'])

merged_df['ex_ret_1'] = merged_df.groupby('Symbol')['Excess Return'].shift(-1)

merged_df.head()

,Symbol,Monthly Returns,Mkt-RF,SMB,HML,RF,Excess Return,ex_ret_1
Date,,,,,,,,
2000-01-31,AAPL,-0.938186,-0.0474,0.0577,-0.0188,0.041,-0.979186,0.061820
2000-01-31,ABT,-0.900699,-0.0474,0.0577,-0.0188,0.041,-0.941699,-0.029564
2000-01-31,ACGL,-0.984903,-0.0474,0.0577,-0.0188,0.041,-1.025903,-0.000265
2000-01-31,ADBE,-0.947964,-0.0474,0.0577,-0.0188,0.041,-0.988964,0.809441
2000-01-31,ADI,-0.911564,-0.0474,0.0577,-0.0188,0.041,-0.952564,0.638818


In [ ]:
"""
A meglévő adatokból válaszd ki a Amazon részvényhez tartozó sorokat (AMZN) és töröld a tickereket tartalmazó oszlopot

amazon_df = merged_df[merged_df['Symbol'] == 'AMZN']
amazon_df = amazon_df.drop(columns=['Symbol'])

7., Készíts egy új LinearRegressionSM elnevezésű osztályt. Definiáld benne a __init__ nevű függvényt, amely bemenetként 2 DataDrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (left_hand_side), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (right_hand_side).

8., Egésztsd ki az osztályt egy fit metódussal, ami meghívja a statsmodels segítségével OLS alapon becsült modellt illeszt az osztály adattagjaira. A becslésből visszakapott modellt mentse le az osztály _model adattagjába. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között.

9., Egészítsd ki az osztályt egy get_params metódussal, ami visszaadja a becsült modellt béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen 'Beta coefficients'

10., Egészítsd ki az osztályt egy get_pvalues metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: P-values for the corresponding coefficients

11., Egészítsd ki az osztályt egy get_wald_test_result metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: F-value: fvalue, p-value: pvalue, ahol az fvalue és pvalue helyére 3 tizedesjegyre kerekítve adja meg a hozzájuk tartozó értékeket.

12., Egészítse ki az osztályt egy get_model_goodness_values metódussal, ami visszadja a módosított R-négyzet, az Akaike és a Bayes információs kritériumok értékét. A visszatérési típus string legyen, a visszaadandó szöveg: Adjusted R-squared: ars, Akaike IC: ak, Bayes IC: by, ahol ars, ak és by helyére 3 tizedesjegyre kerekítve adja meg a hozzájuk tartozó értékeket.

A létrehozott osztályt és a hozzátartozó metódusokat mentsd le a src.linear_regression mappába, a LinearRegressions.py fájlba.
"""

In [ ]:
import pandas as pd
import statsmodels.api as sm

class LinearRegressionSM:
    def __init__(self, left_hand_side, right_hand_side):
        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self._model = None

    def fit(self):
        X = sm.add_constant(self.right_hand_side)
        y = self.left_hand_side
        model = sm.OLS(y, X).fit()
        self._model = model

    def get_params(self):
        if self._model is not None:
            beta_coefficients = self._model.params
            return pd.Series(beta_coefficients, name='Beta coefficients')
        else:
            return None

    def get_pvalues(self):
        if self._model is not None:
            p_values = self._model.pvalues
            return pd.Series(p_values, name='P-values for the corresponding coefficients')
        else:
            return None

    def get_wald_test_result(self, restrictions):
        if self._model is not None:
            wald_test = self._model.wald_test(restrictions)
            f_value = round(wald_test.statistic, 3)
            p_value = round(wald_test.pvalue, 3)
            return f'F-value: {f_value}, p-value: {p_value}'
        else:
            return None
        
    